In [1]:
import openai
import os
import textwrap
import requests
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [3]:
# Test the bing search APi
search = BingSearchAPIWrapper()
search.run("weather in Valencia")

'Current Weather. 8:19 AM.<b> 71° F.</b> RealFeel® 77°. RealFeel Shade™ 70°. Air Quality Fair. Wind N 4 mph. Wind Gusts 5 mph. Sunny More Details. <b>Valencia</b> - BBC <b>Weather</b> <b>Valencia</b> Tonight Low21° Light rain showers and a gentle breeze Tue 4th 31° 21° Wed 5th 30° 22° Thu 6th 30° 22° Fri 7th 30° 20° Sat 8th 31° 20° Sun 9th 32° 21° Mon 10th... Forecast for Spanish towns:<b> València</b> (València/Valencia) - 7-Day<b> weather forecast</b> - Table Weather:<b> València</b> (València/Valencia) - 7-Day<b> weather forecast</b> - Table - State Meteorological Agency - AEMET - Spanish Government <b>Valencia</b> Extended Forecast with high and low temperatures Jul 3 – Jul 9 0.01 Lo:73 Tue, 4 Hi:85 14 Lo:74 Wed, 5 Hi:84 12 Lo:77 Thu, 6 Hi:84 12 Lo:76 Fri, 7 Hi:85 11 Lo:74 Sat, 8 Hi:85 11 Lo:76 Sun, 9 Hi:86 14 Jul 10 – Jul 16 Lo:75 Mon, 10 Hi:90 12 Previsión meteorológica de tres días para <b>Valencia,</b> Comunidad Valenciana,<b></b> España | AccuWeather 2 Aviso amarillo por torme

In [17]:
# memory = ConversationBufferMemory(memory_key="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

search = BingSearchAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

# llm=OpenAI(temperature=0)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.6)
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    verbose=True, 
    memory=memory,
    handle_parsing_errors=True,
)


In [20]:
agent_chain.run(input="Give me basic facts about the company zebra technologies?")



> Entering new  chain...
{
    "action": "Current Search",
    "action_input": "basic facts about Zebra Technologies"
}
Observation: <b>Zebra Technologies ranked #1 among top</b> companies in the<b> technology</b> industry based on trust by customers, investors and employees. As a responsible corporate citizen,<b> Zebra is committed to strong corporate governance practices and environmental and social initiatives that benefit all stakeholders.</b> Acerca de <b>Zebra Technologies</b> Corporation | <b>Zebra</b> Asistencia y descargas Acerca de <b>Zebra</b> Lo conectamos con el rendimiento: donde no hay límites para lo que vendrá. Donde la innovación se une al propósito. Donde la acción se convierte en evolución. Donde la visión impulsa la experiencia. QUIÉNES Somos Zebra Technologies Corporation is<b> an American mobile computing company specializing in technology used to sense, analyze, and act in real time.</b> The company <b>manufactures and sells marking, tracking, and computer pri

'Zebra Technologies Corporation is an American mobile computing company specializing in technology used to sense, analyze, and act in real time. The company manufactures and sells marking, tracking, and computer printing technologies. Zebra Technologies ranked #1 among top companies in the technology industry based on trust by customers, investors, and employees. As a responsible corporate citizen, Zebra is committed to strong corporate governance practices and environmental and social initiatives that benefit all stakeholders.'

In [21]:
agent_chain.memory


ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What should I wear today in Valencia?', additional_kwargs={}, example=False), AIMessage(content='The current weather in Valencia is 71°F with a RealFeel® temperature of 77°F. It is sunny with a gentle breeze. You may want to wear light and comfortable clothing.', additional_kwargs={}, example=False), HumanMessage(content='Is the world in an economic recession?', additional_kwargs={}, example=False), AIMessage(content='The information obtained suggests that there have been recent concerns and discussions about the possibility of a global recession, with some indicators pointing to a weakening global economy and potential downturns in certain regions. However, it is important to note that economic conditions can change rapidly, and it is recommended to consult up-to-date and reliable sources for the most accurate information on the current state of the global economy.', additional_kwargs={}, example=F

In [25]:
agent_chain.run(input="What is the expected growth of Intel?")

OutputParserException: Could not parse LLM output: `Do I need to use a tool? No`

In [13]:
agent_chain.run(input="What is EPS?")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: EPS stands for Earnings Per Share and is a measure of a company's profitability. It is calculated by dividing the company's net income by its total number of outstanding shares. EPS is used to measure the profitability of a company and is an important metric for investors.

> Finished chain.


"EPS stands for Earnings Per Share and is a measure of a company's profitability. It is calculated by dividing the company's net income by its total number of outstanding shares. EPS is used to measure the profitability of a company and is an important metric for investors."

In [18]:
agent_chain.run("Is intel a better stock investment than AMD stock?")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: "Intel vs AMD stock investment"
Observation: <b>Intel ( INTC 0.89%)</b> and<b> Advanced Micro Devices ( AMD -0.62%)</b> have experienced contrasting starts on the stock market in 2022, with the former heading higher despite the sell-off in tech stocks. <b>A Fortune</b> Business Insights report<b> indicated</b> that while<b> the semiconductor market was worth $425.96 billion last year,</b> it<b> is expected to grow to $803.15 billion in 2028.</b> This indicates a compounded... Battlemage aims for market share parity with AMD and I believe by 2024 Intel will<b> overtake AMD for the #2 spot with 12% share</b> to<b> AMD&#39;s 5%</b> - that means a 3% decline for<b> AMD</b> from today&#39;s numbers. <b>Intel will likely spend more than $100B on new</b> fabs<b> in the</b> next 5 years.<b> AMD</b> continues to sub-contract its chip production. See which stock is the better buy. Back then, <b>

"Based on my research, both Intel and AMD have their strengths and weaknesses as stock investments. Intel has a lower price-to-earnings ratio and a higher dividend yield, while AMD has had a better performance in the stock market in recent years. Ultimately, the best choice for you will depend on your specific investment goals and risk tolerance. It's important to do your own research and consult with a financial advisor before making any investment decisions."